In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
PINECONE_API_KEY = "pcsk_6qhT8b_6SZcjLZnYY74HWkUwtBS3vcP6hChwmwmXyZEa26WrHELWpTGjYzxusLqDpM65B8"
PINECONE_API_ENV = "aws-us-east-1"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [6]:
extracted_data

[Document(metadata={'source': 'data\\Michel T.Goodrich ,Roberto Tamassia, Michel H.Gold wasser, Data Structures and Algorithms in Python.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'data\\Michel T.Goodrich ,Roberto Tamassia, Michel H.Gold wasser, Data Structures and Algorithms in Python.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'data\\Michel T.Goodrich ,Roberto Tamassia, Michel H.Gold wasser, Data Structures and Algorithms in Python.pdf', 'page': 2}, page_content='Data Structures and\nAlgorithms in Python\nMichael T. Goodrich\nDepartment of Computer Science\nUniversity of California, Irvine\nRoberto Tamassia\nDepartment of Computer Science\nBrown University\nMichael H. Goldwasser\nDepartment of Mathematics and Computer Science\nSaint Louis University\n'),
 Document(metadata={'source': 'data\\Michel T.Goodrich ,Roberto Tamassia, Michel H.Gold wasser, Data Structures and Algorithms in Python.pdf', 'page': 3}, page_content='VP & PUBLISHER Don 

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 3858


In [9]:
text_chunks

[Document(metadata={'source': 'data\\Michel T.Goodrich ,Roberto Tamassia, Michel H.Gold wasser, Data Structures and Algorithms in Python.pdf', 'page': 2}, page_content='Data Structures and\nAlgorithms in Python\nMichael T. Goodrich\nDepartment of Computer Science\nUniversity of California, Irvine\nRoberto Tamassia\nDepartment of Computer Science\nBrown University\nMichael H. Goldwasser\nDepartment of Mathematics and Computer Science\nSaint Louis University'),
 Document(metadata={'source': 'data\\Michel T.Goodrich ,Roberto Tamassia, Michel H.Gold wasser, Data Structures and Algorithms in Python.pdf', 'page': 3}, page_content='VP & PUBLISHER Don Fowley\nEXECUTIVE EDITOR Beth Lang Golub\nEDITORIAL PROGRAM ASSISTANT Katherine WillisMARKETING MANAGER Christopher RuelDESIGNER Kenji NgiengSENIOR PRODUCTION MANAGER Janis SooASSOCIATE PRODUCTION MANAGER Joyce Poh\nThis book was set in L aTEX by the authors. Printed and bound by Courier Westford.\nThe cover was printed by Courier Westford.\nThis

In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\Nikith\AppData\Local\Temp\ipykernel_15108\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Nikith\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hi I am a robot")
print("Length", len(query_result))

Length 384


In [49]:

from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_6qhT8b_6SZcjLZnYY74HWkUwtBS3vcP6hChwmwmXyZEa26WrHELWpTGjYzxusLqDpM65B8")
index_name = pc.Index("chatbot")


In [43]:
from langchain_pinecone import PineconeVectorStore

In [51]:
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
'''
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)
'''

'"query = "What are Allergies"\n\ndocs=docsearch.similarity_search(query, k=3)\n\nprint("Result", docs)'

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])